In [2]:
import pyterrier as pt
pt.init()

PyTerrier 0.9.2 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [4]:
index = pt.IndexFactory.of('./doc2query_index')
searcher = pt.BatchRetrieve(index)
results = searcher.transform("book about camping")
results[['qid','docno']].head()

/tmp/ipykernel_5979/2270366581.py:3: FutureWarning: .transform() should be passed a dataframe. Use .search() to execute a single query.
  results = searcher.transform("book about camping")


,qid,docno
0,1,11686
1,1,11196
2,1,3102
3,1,10663
4,1,15537


In [7]:
import pandas as pd
import csv
import json
import re
from tqdm import tqdm

def load_data():
  data = []
  with open("booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
      data.append(row)

  book_index = []
  book_id = []
  book_author = []
  book_name = []
  summary = []
  genre = []
  a = 1
  for i in tqdm(data):
      book_index.append(a)
      a = a+1
      book_id.append(i[0])
      book_name.append(i[2])
      book_author.append(i[3])
      genre.append(i[5])
      summary.append(i[6])
  df = pd.DataFrame({'Index': book_index, 'ID': book_id, 'BookTitle': book_name, 'Author': book_author,
                        'Genre': genre, 'Summary': summary})
  def clean_data(df):
    df.isna().sum()
    df = df.drop(df[df['Genre'] == ''].index)
    df = df.drop(df[df['Summary'] == ''].index)
    genres_cleaned = []
    for i in df['Genre']:
        genres_cleaned.append(list(json.loads(i).values()))
    df['Genre'] = genres_cleaned
    
    def clean_summary(text):
      text = re.sub("\'", "", text)
      text = re.sub("[^a-zA-Z]"," ",text)
      text = ' '.join(text.split())
      text = text.lower()
      return text
    df['clean_summary'] = df['Summary'].apply(lambda x: clean_summary(x))
    return df
  df = clean_data(df)
  return df

df = load_data()

16559it [00:00, 19881.58it/s]
100%|██████████| 16559/16559 [00:00<00:00, 522619.21it/s]


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12841 entries, 0 to 16558
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Index          12841 non-null  int64 
 1   ID             12841 non-null  object
 2   BookTitle      12841 non-null  object
 3   Author         12841 non-null  object
 4   Genre          12841 non-null  object
 5   Summary        12841 non-null  object
 6   clean_summary  12841 non-null  object
dtypes: int64(1), object(6)
memory usage: 802.6+ KB


In [21]:
df.loc[df['Index'].isin(results['docno'].astype('int64').tolist())].head()

,Index,ID,BookTitle,Author,Genre,Summary,clean_summary
2,3,986,The Plague,Albert Camus,"[Existentialism, Fiction, Absurdist fiction, N...",The text of The Plague is divided into five p...,the text of the plague is divided into five pa...
35,36,10951,Fahrenheit 451,Ray Bradbury,"[Science Fiction, Children's literature, Specu...",On a rainy night while returning from his job...,on a rainy night while returning from his job ...
57,58,22113,No Logo,Naomi Klein,"[Business, Non-fiction, Economics, Anthropolog...","The book comprises four sections: ""No Space"",...",the book comprises four sections no space no c...
72,73,26476,Rendezvous with Rama,Arthur C. Clarke,"[Hard science fiction, Science Fiction, Specul...",After a major disaster caused by a meteorite ...,after a major disaster caused by a meteorite f...
75,76,28426,Starship Troopers,Robert A. Heinlein,"[Science Fiction, Speculative fiction, Fiction]",Starship Troopers takes place in the midst of...,starship troopers takes place in the midst of ...
